In [5]:
import pandas as pd
import os,shutil
from datetime import datetime, timedelta
import datetime

In [6]:
workbook = pd.ExcelFile('latest_sample_pinged_data.xlsx')

In [7]:
data = pd.read_excel(workbook)

In [8]:
len(data)

52742

In [9]:
#extracts the filtered data to its own variable
def sheet(source,column, string):
    x = source[source[column].str.contains(string,na=False)]
    x = x.sort_values('IP')
    return x
#removes the filtered data from the main variable
def sheet_x(source,column, string):
    x = source[~source[column].str.contains(string,na=False)]
    x = x.sort_values('IP')
    return x

In [10]:
#change last scanned date to_date
# data['Last Scanned Date']=pd.to_datetime(data['Last Scanned Date'], format= '%Y-%m-%d').dt.date

In [11]:
#getting duplicates by date
# duplicates = data[data.sort_values('Last Scanned Date').duplicated('IP', keep='first')]

In [36]:
cms = data[data['Tags'].str.contains('cms|CMS',na=False)]

In [37]:
with pd.ExcelWriter('test_consoles.xlsx') as writer:
    console.to_excel(writer, index=False)

In [12]:
#dropping duplicates keeping last/recent instance
# data =data[~data.sort_values('Last Scanned Date').duplicated('IP', keep='first')]

In [13]:
#older than 30days from last scanned date
# old = data[data['Last Scanned Date'] < datetime.date(2021,1,19)- pd.to_timedelta("30day") ].sort_values('Last Scanned Date' ,ascending=False)

In [14]:
#30 days from last scanned date
# data = data[data['Last Scanned Date'] > datetime.date(2021,1,19)- pd.to_timedelta("30day") ].sort_values('Last Scanned Date' ,ascending=False)

In [15]:
# console = data[data['DNS'].str.contains('console')]

In [16]:
# len(console)

In [17]:
# with pd.ExcelWriter('test_consoles.xlsx') as writer:
#     console.to_excel(writer, index=False)

In [18]:
def cmdb(x):
    ins = x[(x['IP in CMDB']=='IP in CMDB') & (x['DNS in CMDB'] == 'DNS in CMDB')]
    out = x[(x['IP in CMDB'] =='IP NOT in CMDB') & (x['DNS in CMDB'] == 'DNS NOT in CMDB')]
    return ins, out

In [24]:
def writers(frame, title,typed):
    print(len(frame),title,typed)
    with pd.ExcelWriter(f'{title}/{typed}_{title}.xlsx') as create:
        print(title)
        frame.to_excel(create, index=False)
        
    

In [20]:
# filters(data)

In [27]:
print('Sorting "Duplicate IP"')
duplicate = data[data.duplicated(subset=['IP'],keep='last')]
main = data[~data.duplicated(subset=['IP'],keep='last')]
print('Sorting "Available"')
available = sheet(main,'CNAME','available')
main = sheet_x(main,'CNAME','available')
available = available.append(sheet(main,'CNAME','AVAILABLE'))
main = sheet_x(main,'CNAME','AVAILABLE')
print('Sorting "VIP CNAME"')
vip = sheet(main,'CNAME','vip')
main = sheet_x(main,'CNAME','vip')
print('Sorting "Internet Facing Assets"')
pub = sheet(main,'Tags','Internet Facing Assets')
main = sheet_x(main,'Tags','Internet Facing Assets')
print('Sorting "Load Balancers"')
lb = sheet(main,'CNAME','-mip|-vlb|-plb')
main = sheet_x(main,'CNAME','-mip|-vlb|-plb')
#Tags
lb = lb.append(sheet(main,'Tags','loadbalancer'))
main = sheet_x(main,'Tags','loadbalancer')
print('Sorting "Agent"')
agent = sheet(main,'Tracking Method','AGENT')
main = sheet_x(main,'Tracking Method','AGENT')
print('Sorting "Interface"')
interface = sheet(main,'CNAME','interface')
main = sheet_x(main,'CNAME','interface')
print('Sorting "links"')
link = sheet(main,'CNAME','-link')
main = sheet_x(main,'CNAME','-link')
print('Sorting "None"')
blank = sheet(main,'CNAME','None|none')
main = sheet_x(main,'CNAME','None|none')
print('Sorting "DevOps"')
devops = sheet(main,'Tags','BU:DevOps')
main = sheet_x(main,'Tags','BU:DevOps')
print('Sorting "Duplicate CNAME"')
duplicate = duplicate.append(main[main.duplicated(subset=['CNAME'],keep='last')])
main = main[~main.duplicated(subset=['CNAME'],keep='first')]
print('Sorting "LS"')
ls = sheet(main,'CNAME','-ls')
main = sheet_x(main,'CNAME','-ls')
print('Sorting "CY"')
cy = sheet(main,'CNAME','cy')
main = sheet_x(main,'CNAME','cy')
print('Sorting "bo1"')
bo1 = sheet(main,'CNAME','BO1|bo1')
main = sheet_x(main,'CNAME','BO1|bo1')
#Tags
bo1 = bo1.append(sheet(main,'Tags','BO1|bo1'))
main = sheet_x(main,'Tags','BO1|bo1')
print('Sorting "Bronto"')
bronto = sheet(main,'CNAME','Bronto|bronto')
main = sheet_x(main,'CNAME','Bronto|bronto')
#tags
bronto = bronto.append(sheet(main,'Tags','Bronto|bronto'))
main = sheet_x(main,'Tags','Bronto|bronto')
print('Sorting "OpenAir"')
openair = sheet(main,'Tags','openair|OpenAir')
main = sheet_x(main,'Tags','openair|OpenAir')
print('Sorting "OCI"')
oci = sheet(main,'CNAME','OCI|oci')
main = sheet_x(main,'CNAME','OCI|oci')
#tags
oci = oci.append(sheet(main,'Tags','OCI|oci'))
main = sheet_x(main,'Tags','OCI|oci')
print('Sorting "Payroll"')
payroll = sheet(main,'Tags','payroll|Payroll')
main = sheet_x(main,'Tags','payroll|Payroll')
print('Sorting "VPN"')
vpn = sheet(main,'CNAME','vpn|ovpn')
main = sheet_x(main,'CNAME','vpn|ovpn')
#tags
vpn = vpn.append(sheet(main,'Tags','vpn|ovpn'))
main = sheet_x(main,'Tags','vpn|ovpn')
print('Sorting "Console"')
console = sheet(main,'CNAME','console')
main = sheet_x(main,'CNAME','console')
#tags
console = console.append(sheet(main,'Tags','console'))
main = sheet_x(main,'Tags','console')
print('Sorting "NoUse"')
nouse = sheet(main,'CNAME','nouse')
main = sheet_x(main,'CNAME','nouse')
print('Sorting "VM"')
vm = sheet(main,'CNAME','snap|\.f\.|\.m\.')
main = sheet_x(main,'CNAME','snap|\.f\.|\.m\.')
print('Sorting "PDU"')
pdu = sheet(main,'CNAME','pdu-')
main = sheet_x(main,'CNAME','pdu-')
#IP address filter
pdu = pdu.append(sheet(main,'IP','^10\..*\..*\.24[0-8]'))
main = sheet_x(main,'IP','^10\..*\..*\.24[0-8]')
print('Finished sorting filters')
filter_frame = [main,lb,devops,link, blank, duplicate, agent, ls, cy, pdu, pub, bo1, bronto, openair, oci, payroll, console, vpn, nouse, vm,interface, available,vip]
filter_title = ['main','Load Balancer','DevOps','Link', 'None', 'Duplicate', 'Agent', 'LS', 'CY', 'PDU', 'Internet Facing Asset', 'BO1', 'Bronto', 'OpenAir', 'OCI'
                , 'Payroll', 'Console', 'VPN', 'NoUse', 'VM','Interface', 'Available', 'VIP_NAT']


Sorting "Duplicate IP"
Sorting "Available"
Sorting "VIP CNAME"
Sorting "Internet Facing Assets"
Sorting "Load Balancers"
Sorting "Agent"
Sorting "Interface"
Sorting "links"
Sorting "None"
Sorting "DevOps"
Sorting "Duplicate CNAME"
Sorting "LS"
Sorting "CY"
Sorting "bo1"
Sorting "Bronto"
Sorting "OpenAir"
Sorting "OCI"
Sorting "Payroll"
Sorting "VPN"
Sorting "Console"
Sorting "NoUse"
Sorting "VM"
Sorting "PDU"
Finished sorting filters


In [28]:
for frame, title in zip(filter_frame, filter_title):
    if len(frame) >0:
        if os.path.exists(title):
            shutil.rmtree(title)
            os.mkdir(title)
        else:
            os.mkdir(title)
        
        ins,out=cmdb(frame)
        if len(ins)>0:
#             print (f'{title}_ins,{len(ins)}')
            writers(ins, title,'in_cmdb')
        else:
            pass

        if len(out) >0:
#             print (f'{title}_out,{len(out)}')
            writers(out, title,'out_cmdb')
        else:
            pass

5657 main in_cmdb
main
514 main out_cmdb
main
36 Load Balancer in_cmdb
Load Balancer
591 Load Balancer out_cmdb
Load Balancer
129 DevOps out_cmdb
DevOps
746 Link out_cmdb
Link
2 None in_cmdb
None
15677 None out_cmdb
None
7 Duplicate in_cmdb
Duplicate
2563 Duplicate out_cmdb
Duplicate
2886 Agent in_cmdb
Agent
354 Agent out_cmdb
Agent
698 LS in_cmdb
LS
37 LS out_cmdb
LS
76 CY in_cmdb
CY
122 CY out_cmdb
CY
1106 PDU in_cmdb
PDU
49 PDU out_cmdb
PDU
996 Internet Facing Asset out_cmdb
Internet Facing Asset
3 BO1 in_cmdb
BO1
3 BO1 out_cmdb
BO1
273 Bronto out_cmdb
Bronto
10 OCI in_cmdb
OCI
11 OCI out_cmdb
OCI
1 Payroll out_cmdb
Payroll
29 Console in_cmdb
Console
4683 Console out_cmdb
Console
20 VPN in_cmdb
VPN
5035 VPN out_cmdb
VPN
848 VM in_cmdb
VM
101 VM out_cmdb
VM
2755 Interface out_cmdb
Interface
4740 Available out_cmdb
Available
50 VIP_NAT out_cmdb
VIP_NAT


In [ ]:

    
    
        
# with pd.ExcelWriter('test.xlsx') as writer:
#     main.to_excel(writer,sheet_name='main', index=False)
#     agent.to_excel(writer,sheet_name='AGENT', index=False)
#     netops.to_excel(writer,sheet_name='netops', index=False)
#     nscore.to_excel(writer,sheet_name='nscore', index=False)
#     bronto.to_excel(writer,sheet_name='bronto', index=False)
#     openair.to_excel(writer,sheet_name='openair', index=False)
#     payroll.to_excel(writer,sheet_name='payroll', index=False)
#     devops.to_excel(writer,sheet_name='devops', index=False)
#     pub.to_excel(writer,sheet_name='Internet Facing Assets', index=False)
#     nspos.to_excel(writer,sheet_name='NSPOS', index=False)
#     test_asset.to_excel(writer,sheet_name='Test Asset', index=False)
#     legacy.to_excel(writer,sheet_name='LegacyCorp', index=False)
    
    
    
    
    
    
    
    
    